In [1]:
import re 
import urllib.request as urllib_request
import pandas as pd
from urllib.error import URLError, HTTPError
from utils.helpers import *
from utils.functions import *
pd.set_option('display.max_columns', 100)

In [2]:
URL = 'https://fiis.com.br/lista-de-fundos-imobiliarios/'
soup = create_bsoup_object(URL)
tickers = get_tickers(soup)

In [3]:
cards = []
for ticker in tickers:
    url = 'https://fiis.com.br/' + ticker
    soup = create_bsoup_object(url)
    items = soup.find_all(class_ = 'value')
    infos = []
    card = {}
    for item in items:
        infos.append(item.get_text())
    card['ticker'] = ticker.upper()
    admin = soup.findAll("div", {"class": "text-wrapper"})
    card['administrador'] = re.sub("ADMINISTRADOR|[0-9].*[0-9]", "", admin[0].get_text())
    card['dividend_yield'] = infos[0]
    card['ultimo_rendimento'] = infos[1]
    card['patrimonio_liquido'] = infos[2]
    card['valor_patrimonial_por_cota'] = infos[3]
    card['telefone'] = infos[4]
    card['email'] = infos[5]
    card['site'] = infos[6]
    card['name'] = infos[7]
    card['tipo_FII'] = infos[8]
    card['tipo_ANBIMA'] = infos[9]
    card['registro_CVM'] = infos[10]
    card['numero_cotas'] = infos[11]
    card['numero_cotistas'] = infos[12]
    card['CNPJ'] = infos[13]
    card['cotacao'] = infos[14]
    card['min_52_weeks'] = infos[15]
    card['max_52_weeks'] = infos[16]
    card['return_12_months'] = infos[17]
        
    cards.append(card)

In [4]:
## Create dataframe with the results
dataset = pd.DataFrame(cards)

In [5]:
COLUMNS_LIST = ['ultimo_rendimento',
                'valor_patrimonial_por_cota',
                'numero_cotas',
                'numero_cotistas',
                'cotacao',
                'min_52_weeks',
                'max_52_weeks']
dataset = convert_string_to_float(dataset, COLUMNS_LIST)
dataset = convert_percentages_to_float(dataset, ['dividend_yield', 'return_12_months'])
dataset['registro_CVM'] = dataset['registro_CVM'].apply(lambda x: convert_to_datetime(x))
dataset["p_vp"] = dataset['cotacao'] / dataset['valor_patrimonial_por_cota']

In [6]:
dataset.head()

,ticker,Administrador,dividend_yield,ultimo_rendimento,patrimonio_liquido,valor_patrimonial_por_cota,telefone,email,site,name,tipo_FII,tipo_ANBIMA,registro_CVM,numero_cotas,numero_cotistas,CNPJ,cotacao,min_52_weeks,max_52_weeks,return_12_months,p_vp
0,ABCP11,Rio Bravo Investimentos,0.0089,0.65,"R$1,12 B",91.83,(11) 3509-6600,[email protected],www.riobravo.com.br,FII ABC IMOB,Tijolo: Shoppings,Renda Gestão Passiva,1996-05-15,12203832.0,19040.0,01.201.140/0001-90,72.80,60.01,78.27,0.0502,0.792769
1,AFHI11,BTG Pactual,0.0137,1.35,"R$172,46 M",96.47,21 3262-9600,[email protected],www.btgpactual.com,FII AFHI CRI,Papel: CRIs,Títulos e Valores Mobiliários Gestão Ativa,2020-10-20,1787671.0,7700.0,36.642.293/0001-58,98.25,88.00,103.45,0.0679,1.018451
2,AFOF11,BRL Trust DTVM,0.0128,1.14,"R$70,55 M",102.07,(11)31330350,[email protected],www.brltrust.com.br,FII ALIANZFF,Papel: Fundo de Fundos,Títulos e Valores Mobiliários Gestão Ativa,2020-01-22,691192.0,2584.0,34.847.063/0001-08,88.80,75.05,300.00,0.6036,0.869991
3,AIEC11,MAF DTVM,0.0091,0.72,"R$491,83 M",101.93,3223-7700,[email protected],www.modal.com.br,FII AUTONOMY,Tijolo: Escritórios,Renda Gestão Ativa,2020-08-24,4824987.0,6762.0,35.765.826/0001-26,78.99,67.50,88.00,0.1023,0.774944
4,ALMI11,BTG Pactual,0.0004,0.37,"R$250,49 M",2253.05,21 3262-9600,[email protected],www.btgpactual.com,FII TORRE AL,Tijolo: Escritórios,Renda Gestão Passiva,2006-01-03,111177.0,3114.0,07.122.725/0001-00,932.03,830.00,1300.00,0.1823,0.413675
